<a href="https://colab.research.google.com/github/Ranjali12/DeeployCV/blob/main/assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import cv2
import torch
from ultralytics import YOLO


In [8]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s') # or yolov5m, yolov5l, yolov5x, custom
# model = YOLO('yolov8n.pt')  # Load the model
model.to('cpu')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-21 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [9]:
video_path = 0
cap = cv2.VideoCapture(video_path)


In [11]:
# main code execution
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame (OpenCV uses BGR, YOLOv5 uses RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform inference
    results = model(frame_rgb)

    # Render results on the frame
    annotated_frame = results.render()[0]

    # Convert back to BGR for OpenCV display
    annotated_frame = cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR)

    # Display the frame
    cv2.imshow('YOLOv5 Object Detection', annotated_frame)

    # Break on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [12]:
import cv2
from ultralytics import YOLO
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [13]:
yolo_model = YOLO('yolov8n.pt')  # Load the model
yolo_model.to('cpu') # set to cpu

100%|██████████| 6.25M/6.25M [00:00<00:00, 76.1MB/s]


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [14]:
def locate_flag_region_with_yolo(image_path):
    # Use YOLOv8 to detect objects in the image
    results = yolo_model.predict(source=image_path, conf=0.25)
    detections = results[0].boxes.xyxy.numpy()  # Get bounding boxes (x_min, y_min, x_max, y_max)

    if len(detections) == 0:
        return None, None  # No flag detected
    largest_box = max(detections, key=lambda box: (box[2] - box[0]) * (box[3] - box[1]))

    x_min, y_min, x_max, y_max = map(int, largest_box)
    image = Image.open(image_path)
    flag_region = image.crop((x_min, y_min, x_max, y_max))
    return flag_region, (x_min, x_max, y_min, y_max)

In [15]:
def identify_flag_with_sobel(image_path):
    flag_region, region_coords = locate_flag_region_with_yolo(image_path)
    if flag_region is None:
        return "No flag detected", None

    # Resize the flag region
    resized_flag = flag_region.resize((100, 60))
    flag_array = np.array(resized_flag.convert("L"))  # Convert to grayscale

    # Apply Sobel edge detection
    sobel_x = cv2.Sobel(flag_array, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(flag_array, cv2.CV_64F, 0, 1, ksize=3)
    sobel_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)

    # Calculate edge strength for top and bottom halves
    height = flag_array.shape[0]
    top_half = sobel_magnitude[:height // 2, :]
    bottom_half = sobel_magnitude[height // 2:, :]
    top_strength = np.sum(top_half)
    bottom_strength = np.sum(bottom_half)

    # Determine flag based on edge strength
    if top_strength > bottom_strength:  # Top has more edges (red for Indonesia)
        return "Indonesia", region_coords
    elif bottom_strength > top_strength:  # Bottom has more edges (red for Poland)
        return "Poland", region_coords
    else:
        return "Unknown flag", region_coords

In [16]:
def identify_flag(image_path):
    # Step 1: Locate the flag region with YOLOv8
    flag_region, region_coords = locate_flag_region_with_yolo(image_path)
    if flag_region is None:
        return "No flag region detected", None

    # Step 2: Resize the flag region to a standard size
    resized_flag = flag_region.resize((100, 60))

    # Step 3: Convert the flag to an RGB array
    rgb_flag = np.array(resized_flag)

    # Step 4: Split into top and bottom halves
    height = rgb_flag.shape[0]
    top_half = rgb_flag[:height // 2, :, :]
    bottom_half = rgb_flag[height // 2:, :, :]

    # Step 5: Calculate mean red intensity of each half
    top_red_mean = np.mean(top_half[:, :, 0])  # Red channel of top half
    bottom_red_mean = np.mean(bottom_half[:, :, 0])  # Red channel of bottom half

    # Step 6: Determine flag based on red intensity comparison
    if top_red_mean > bottom_red_mean:  # Top is red, bottom is white
        return "Indonesia", region_coords
    elif bottom_red_mean > top_red_mean:  # Top is white, bottom is red
        return "Poland", region_coords
    else:
        return "Unknown flag", region_coords

In [ ]:
def show_images_in_grid(image_paths):
    num_images = len(image_paths)

    # Create a grid for displaying images
    fig, axes = plt.subplots(num_images, 2, figsize=(10, num_images * 3))

    for i, image_path in enumerate(image_paths):
        result, region_coords = identify_flag(image_path) # Change this line to use a different methods

        # Display the image with detected region
        image = Image.open(image_path)
        axes[i, 0].imshow(image)
        axes[i, 0].axis('off')
        axes[i, 0].set_title(f"{result}")

        if region_coords:
            x_min, x_max, y_min, y_max = region_coords
            axes[i, 0].add_patch(
                plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                              linewidth=2, edgecolor='red', facecolor='none')
            )

        # Show the cropped region (flag region) on the right side
        flag_region, _ = locate_flag_region_with_yolo(image_path)
        if flag_region is not None:
            axes[i, 1].imshow(flag_region, cmap='gray')
            axes[i, 1].axis('off')
            axes[i, 1].set_title("Flag Region")

    plt.tight_layout()
    plt.show()